In [ ]:
import sys, os, datetime, hashlib, hmac,requests, logging,urllib,urllib3
from urllib.parse import urlparse
import boto3,botocore
import http.client as http_client
from  nsdf.s3 import S3GuessEndPoint
urllib3.disable_warnings()

profile_name,path="wasabi"       ,"/nsdf/nsdf-test/visus.idx"
profile_name,path="cloudbank"    ,"/nsdf/nsdf-test/visus.idx"
profile_name,path="sealstorage"  ,"/api/v0/s3/utah/nsdf/nsdf-test/visus.idx"
profile_name,path="chpc"         ,"/nsdf/nsdf-test/visus.idx"
profile_name,path="mghpcc"       ,"/vpascuccibucket1/nsdf/nsdf-test/visus.idx"
profile_name,path="ucsd"         ,"/nsdf/nsdf-test/visus.idx"

endpoint_url=S3GuessEndPoint(profile_name)
session = boto3.Session(profile_name=profile_name)
region=session.region_name
credentials = session.get_credentials()
access_key=credentials.access_key
secret_key=credentials.secret_key

url=urlparse(endpoint_url).scheme + "://" + urlparse(endpoint_url).netloc + path
print(url)
print(endpoint_url)
assert url.startswith(endpoint_url) 
assert url[len(endpoint_url)]=="/"
bucket,key=url[len(endpoint_url)+1:].split("/",maxsplit=1)

debug_network=False
if debug_network:
	http_client.HTTPConnection.debuglevel = 1
	logging.basicConfig()
	logging.getLogger().setLevel(logging.DEBUG)
	requests_log = logging.getLogger("requests.packages.urllib3")
	requests_log.setLevel(logging.DEBUG)

# if you want to debug and compare step by step
if False:
	print("# BOTO3********* \n")
	print("#See https://docs.aws.amazon.com/general/latest/gr/sigv4-signed-request-examples.html")
	client = session.client("s3", config= botocore.config.Config(signature_version='s3v4'), endpoint_url=endpoint_url)
	signed = client.generate_presigned_url('get_object',Params={'Bucket': bucket,'Key': key})
	response=requests.get(signed, verify=False)
	print(response.text)

def GetSignatureKey(key, dateStamp, regionName, serviceName):
	def Sign(key, msg): return hmac.new(key, msg.encode('utf-8'), hashlib.sha256).digest()
	kDate    = Sign(('AWS4' + key).encode('utf-8'), dateStamp)
	kRegion  = Sign(kDate, regionName)
	kService = Sign(kRegion, serviceName)
	kSigning = Sign(kService, 'aws4_request')
	return kSigning

def MakeHeaders(d):
  return "&".join([key + "=" + urllib.parse.quote(value, safe='') for key,value in d.items()])

def HexDigest(s):
	ret=""
	for ch in s: ret+=f"{ch:02x}"
	return ret

def S3V4(debug=None, algorithm="AWS4-HMAC-SHA256", method="GET"):

	if debug:
		amzdate=debug
	else:
		amzdate= datetime.datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')
  
	datestamp =amzdate[0:8]
 
	from collections import OrderedDict
	
	# for computing the signature the order is ASC
	headers=OrderedDict({
	  	"X-Amz-Algorithm" : algorithm,
	    "X-Amz-Credential" : f"{access_key}/{datestamp}/{region}/s3/aws4_request",
	    "X-Amz-Date" : amzdate,
	    "X-Amz-Expires" : '3600',
	    "X-Amz-SignedHeaders" : "host"
	})
	parsed_url=urllib3.util.parse_url(url)
	canonical_request = "\n".join([method,parsed_url.path,MakeHeaders(headers),(f"host:{parsed_url.hostname}" + '\n'),"host","UNSIGNED-PAYLOAD"])
	credential_scope="/".join([datestamp,region,"s3",'aws4_request'])
	string_to_sign = "\n".join([algorithm,amzdate,credential_scope,hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()])
	signing_key = GetSignatureKey(secret_key, datestamp, region, "s3")
	signature = hmac.new(signing_key, string_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()

	if True:
		print("# amzdate",amzdate)
		print("# canonical_request",canonical_request)
		print("# credential_scope",credential_scope)
		print("# string_to_sign",string_to_sign)
		print("# signing_key",HexDigest(signing_key))
		print("# signature",signature)
  
	headers["X-Amz-Signature"]=signature
	return url,headers

url,headers=S3V4()  # "20220921T200927Z"

# note: headers must be in the url
url,headers = url + "?" + MakeHeaders(headers),None
response=requests.get(url,headers=headers,verify=False)
print(response.text)

#resp=urllib3.PoolManager(cert_reqs='CERT_NONE').request('GET', url, headers=headers)
#print(resp.data.decode('utf-8'))